# Example code for fragmentomic feature extraction

The purpose of this notebook is to provide insights into how different fragementomic features were extracted from aligned cfDNA sequencing data for our cfDNA GWAS study.

The following code snippets make use of the bioinformatics commandline tools: samtools and bedtools. The python code makes use of pandas and pysam.

The code is meant to provide insight into how features were extracted and is not meant to be run out of the box.

The actual code that was run as part of a multiple independent workflows.

In [131]:
import pysam
import numpy as np
import pandas as pd

# Nucleotide Cleave-site motif diversity

In [5]:

cramfilename="test.cram"
reference="hg38flat.fa"

cram=pysam.AlignmentFile(cramfilename,reference_filename=reference)
fasta=pysam.FastaFile(reference)

k=8

revcomptable = str.maketrans("acgtACGT","tgcaTGCA")

# n=int(wildcards.samplen) if wildcards.samplen!="ALL" else None

kmers=[]
d={}
for i in range(4**k):
    s=""
    for j in range(k):
        s+="ACGT"[int(i/(4**(k-j-1)))%4]

    rcs=s.translate(revcomptable)[::-1]

    if s <= rcs:
        kmers.append(s)
        d[s]=0

i=0
for read in cram:
    if not read.is_unmapped and not read.is_duplicate and read.mapq==60 and read.reference_start>int(k/2) and read.reference_end<cram.get_reference_length(read.reference_name)-int(k/2):

        if read.is_reverse:
            s=fasta.fetch(read.reference_name,int(read.reference_end-k/2),int(read.reference_end+k/2)).upper()
        else:
            s=fasta.fetch(read.reference_name,int(read.reference_start-k/2),int(read.reference_start+k/2)).upper()

        if 'N' not in s:
            try:
                rcs=s.translate(revcomptable)[::-1]

                d[s if s<rcs else rcs]+=1
                i+=1
            except KeyError: #skip when reads have other characters than ACGT
                print("Err",s)
                pass

[E::cram_index_load] Could not retrieve index file for 'test.cram'


The diversity index is then calculated as follows:

In [102]:
cleavesitemotifdist=np.array(list(d.values()))+1

In [103]:
np.sum( (-(cleavesitemotifdist/cleavesitemotifdist.sum()) * np.log(cleavesitemotifdist/cleavesitemotifdist.sum())) / (np.log(len(cleavesitemotifdist))) )

0.9388584615800628

# Purine Pyrimidine cleave-site motif diversity

Calculate the purine/pyrimidine cleave-site motif diversity from the nucleotide cleave-site motif frequencies, using the following helper functions:

In [83]:
#Collapse nucleotide sequence to Purine/Pyrimidine sequence
def nuc2purpyr(s):
    n2p={'A':'R','G':'R','C':'Y','T':'Y'} #R=purine / Y=Pyrimidine
    return "".join([n2p[c] for c in s])

def allk(k,onlylexsmallest=False):
    kmers=[]
    for i in range(4**k):
        s=""
        for j in range(k):
            s+="ACGT"[int(i/(4**(k-j-1)))%4]
        if onlylexsmallest:
            if s<=revcomp(s):
                kmers.append(s)
        else:
            kmers.append(s)
    return kmers

def allkp(k,onlylexsmallest=False):
    kpmers=[]
    for i in range(2**k):
        s=""
        for j in range(k):
            s+="RY"[int(i/(2**(k-j-1)))%2]
        if onlylexsmallest:
            if s<=revcomp(s):
                kpmers.append(s)
        else:
            kpmers.append(s)
    return kpmers

def PPrefseqends(seqends,k):
    seqendsk=len(seqends.columns[0])
    kmers=allkp(k)
    df=pd.DataFrame(columns=kmers)
    for ki in kmers:
        df[ki]=seqends[seqends.columns[seqends.columns.map(nuc2purpyr).str.slice(int((seqendsk-k)/2),seqendsk-int(np.ceil((seqendsk-k)/2)))==ki]].sum(axis=1)
    return df

In [80]:
dfpp=PPrefseqends(pd.DataFrame.from_dict(d,orient='index').T,8)

Calculate diversity score in a similar manner:

In [90]:
(dfpp).apply(lambda f: np.sum( (-(f/f.sum()) * np.log(f/f.sum())) /(np.log(f.shape[0])) ),axis=1)

0    0.949768
dtype: float64

# Fragment size diversity

To calculate the fragment size diversity index we used a subset of the output from the stats subcommand of samtools. Which was run using the following parameters

In [93]:
!samtools stats test.cram -F1024 --reference hg38flat.fa > test.stats

/bin/bash: samtools: command not found


In [97]:
insertsizes={}

for i in range(60,600):
    insertsizes[i]=0

with open("test.stats",'rt') as statfile:
    for line in statfile:
        if line.startswith("IS"):
            cols=line.rstrip().split("\t")
            insertsize=int(cols[1])
            number=int(cols[2]) #use all pair orientations?!
            if insertsize>=60 and insertsize<=600:
                insertsizes[insertsize]=number


In [106]:
fsdist=np.array(list(insertsizes.values()))+1

In [107]:
np.sum( (-(fsdist/fsdist.sum()) * np.log(fsdist/fsdist.sum())) / (np.log(len(fsdist))) )

0.7598509172946986

# Bincount diversity

To calculate the bin count diversity index we used bedtools to create 50kbp (and 1Mbp) windows across the human genome, ignoring the short arms of acrocentric chromosomes, Par regions and other complicated regions of the genome:

Todo so, it needs hg38.cytoBand.txt and hg38flat.chrom.sizes

In [111]:
!bedtools makewindows -g hg38flat.chrom.sizes -w 50000 | grep -v chrUn | grep -v random | grep -v alt | bedtools intersect -v - a - -b <(cat hg38.cytoBand.txt | grep -e acen -e stalk -e gvar) |  bedtools intersect -v -a - -b chrYpar.bed > windows.bed

/bin/bash: bedtools: command not found
/bin/bash: bedtools: command not found
/bin/bash: bedtools: command not found


In [114]:
!samtools view --reference hg38flat.fa -b -q 1 -F 1024 test.cram | bedtools coverage -sorted -iobuf 2G  -a windows.bed -b stdin > counts.bed

/bin/bash: samtools: command not found
/bin/bash: bedtools: command not found


Now just load the counts

In [127]:
countdist=[]
with open("counts.bed") as counts:
    for line in counts:
        countdist.append(int(line.split("\t")[3]))

And similarly calculate the diversity index

In [129]:
countdist=np.array(list(countdist))+1

In [130]:
np.sum( (-(countdist/countdist.sum()) * np.log(countdist/countdist.sum())) / (np.log(len(countdist))) )

0.9962795252691058